___
## Ciência dos Dados - PROJETO 3 - INSPER

___
#### Victor Arruda Niubó

#### Giovanni Pertence Cardoso dos Santos

#### Gustavo Schlieper Tessitore

#### Leonardo Cesari Costa
___

# Previsão da quantidade de views

___

## 1. INTRODUÇÃO

O objetivo do projeto é estimar a quantidade de visualizações que um vídeo terá com base no título, horário de publicação e as tags atribuidas aos vídeos.

___
## 2. MINERANDO DADOS e CARACTERÍSTICAS DO DATASET

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats
import itertools
import math
import json
import os
from scipy.stats import norm

In [3]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\gugat\Desktop\Insper DS 2018.1\2.1 semestre\Dados\cdados_dp\P3


### BASE DE DADOS

In [4]:
# Base de dados dos videos em destaque do youtube US
you_data = pd.read_csv("USvideos.csv")
# jason que contém a relação id - categoria
ids = json.loads(open('US_category_id.json').read())

# filtro das variáveis de interesse
you_analysis = you_data.loc[you_data.views|you_data.title|you_data.tags|you_data.category_id,\
                           ['views', 'title', 'tags', 'category_id']]

you_analysis.head()

,views,title,tags,category_id
0,748374,WE WANT TO TALK ABOUT OUR MARRIAGE,SHANtell martin,22
1,2418783,The Trump Presidency: Last Week Tonight with J...,last week tonight trump presidency|last week t...,24
2,3191434,"Racist Superman | Rudy Mancuso, King Bach & Le...",racist superman|rudy|mancuso|king|bach|racist|...,23
3,343168,Nickelback Lyrics: Real or Fake?,rhett and link|gmm|good mythical morning|rhett...,24
4,2095731,I Dare You: GOING BALD!?,ryan|higa|higatv|nigahiga|i dare you|idy|rhpc|...,24


In [5]:
# separação da string das tags em uma lista de strings
tags_split = you_analysis.tags.str.split(pat = '|')
you_analysis['tags_split'] = tags_split
you_analysis.head()

,views,title,tags,category_id,tags_split
0,748374,WE WANT TO TALK ABOUT OUR MARRIAGE,SHANtell martin,22,[SHANtell martin]
1,2418783,The Trump Presidency: Last Week Tonight with J...,last week tonight trump presidency|last week t...,24,"[last week tonight trump presidency, last week..."
2,3191434,"Racist Superman | Rudy Mancuso, King Bach & Le...",racist superman|rudy|mancuso|king|bach|racist|...,23,"[racist superman, rudy, mancuso, king, bach, r..."
3,343168,Nickelback Lyrics: Real or Fake?,rhett and link|gmm|good mythical morning|rhett...,24,"[rhett and link, gmm, good mythical morning, r..."
4,2095731,I Dare You: GOING BALD!?,ryan|higa|higatv|nigahiga|i dare you|idy|rhpc|...,24,"[ryan, higa, higatv, nigahiga, i dare you, idy..."


In [6]:
# Junta todas as strings em uma lista só para pder fazer um value_counts() e criar um
# dataframe com as tags e as frquencias de cada uma
merged = list(itertools.chain(*you_analysis.tags_split))
contador = pd.DataFrame({'col':merged})
limpino = contador[contador.col != '[none]']
dfl = limpino.col.value_counts().rename_axis('tags').reset_index(name='counts')
dfl = dfl[(dfl['counts'])> 500]
dfl

,tags,counts
0,funny,3603
1,comedy,2931
2,how to,1604
3,music,1302
4,Pop,1280
5,2018,1275
6,humor,1185
7,food,1159
8,science,1111
9,review,1005


In [9]:
# Separa as linhas que contém certa tag
contador = 0
for e in dfl.tags:
    var_df = you_analysis[you_analysis['tags'].str.contains(e)]
    var_df[e + '_Freq'] = dfl.counts[contador]

    not_var_df = you_analysis[~you_analysis['tags'].str.contains(e)]
    not_var_df[e + '_Freq'] = 0
    
    contador += 1
    
    toto = pd.concat([var_df, not_var_df])
    you_analysis[e + '_Freq'] = toto[e + '_Freq']

you_analysis

C:\Users\gugat\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\gugat\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,views,title,tags,category_id,tags_split,funny_Freq,comedy_Freq,how to_Freq,music_Freq,Pop_Freq,...,music video_Freq,talk show_Freq,documentary_Freq,nba_Freq,movies_Freq,Funny_Freq,reaction_Freq,technology_Freq,family_Freq,show_Freq
0,748374,WE WANT TO TALK ABOUT OUR MARRIAGE,SHANtell martin,22,[SHANtell martin],0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2418783,The Trump Presidency: Last Week Tonight with J...,last week tonight trump presidency|last week t...,24,"[last week tonight trump presidency, last week...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3191434,"Racist Superman | Rudy Mancuso, King Bach & Le...",racist superman|rudy|mancuso|king|bach|racist|...,23,"[racist superman, rudy, mancuso, king, bach, r...",0,0,0,1302,0,...,557,0,0,0,0,0,0,0,0,0
3,343168,Nickelback Lyrics: Real or Fake?,rhett and link|gmm|good mythical morning|rhett...,24,"[rhett and link, gmm, good mythical morning, r...",3603,2931,0,1302,0,...,0,0,0,0,0,0,0,0,0,0
4,2095731,I Dare You: GOING BALD!?,ryan|higa|higatv|nigahiga|i dare you|idy|rhpc|...,24,"[ryan, higa, higatv, nigahiga, i dare you, idy...",3603,2931,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,119180,2 Weeks with iPhone X,ijustine|week with iPhone X|iphone x|apple|iph...,28,"[ijustine, week with iPhone X, iphone x, apple...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,2103417,Roy Moore & Jeff Sessions Cold Open - SNL,SNL|Saturday Night Live|SNL Season 43|Episode ...,24,"[SNL, Saturday Night Live, SNL Season 43, Epis...",3603,2931,0,1302,0,...,0,0,0,0,0,0,0,0,0,0
7,817732,5 Ice Cream Gadgets put to the Test,5 Ice Cream Gadgets|Ice Cream|Cream Sandwich M...,28,"[5 Ice Cream Gadgets, Ice Cream, Cream Sandwic...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,826059,The Greatest Showman | Official Trailer 2 [HD]...,Trailer|Hugh Jackman|Michelle Williams|Zac Efr...,1,"[Trailer, Hugh Jackman, Michelle Williams, Zac...",0,0,0,1302,0,...,0,0,0,0,0,0,0,0,0,513
9,256426,Why the rise of the robots won’t mean the end ...,vox.com|vox|explain|shift change|future of wor...,25,"[vox.com, vox, explain, shift change, future o...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### ANÁLISE DESCRITIVA

___
## C. MODELOS DE PREDIÇÃO

o MODELO DE PREDIÇÃO PELA MÉDIA (Sem uso de variável explicativa).

o MODELO DOS K VIZINHOS MAIS PRÓXIMOS (K-Nearest Neighbors Regression)

o MODELO DE REGRESSÃO LINEAR (Multiple Linear Regression)

o MODELO DE ÁRVORES DE REGRESSÃO (Decision Tree Regression)

___
## D. PROCESSO E ESTATÍSTICAS DE VALIDAÇÃO

[Esse item depende dos resultados das modelagens anteriores! Organize-os aqui de forma clara!]

___
## E. CONCLUSÃO

___
## F. REFERÊNCIAS BIBLIOGRÁFICAS